<a href="https://colab.research.google.com/github/ConstantinVictorBeatErtel/AutomatedDCF/blob/main/DCF_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


import requests
import numpy as np
import pandas as pd
!pip install certifi
import certifi

symbol= (input(f"Enter your company ticker "))

# Define the API endpoint and parameters
api_url = 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={}&apikey=P7M6C5PE71GNLCKN'.format(symbol)

# Get the income statement data
response = requests.get(api_url)
income_statement = response.json()

api_url_2 = 'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={}&apikey=P7M6C5PE71GNLCKN'.format(symbol)

# Get the income statement data
response_2 = requests.get(api_url_2)
balance_sheet = response_2.json()



Enter your company ticker ADBE


In [26]:
import datetime

fye_year_date = datetime.datetime.strptime(income_statement.get('annualReports', [])[0].get('fiscalDateEnding'), '%Y-%m-%d').date()
ytd_year_date = datetime.datetime.strptime(income_statement.get('quarterlyReports', [])[0].get('fiscalDateEnding'), '%Y-%m-%d').date()

if ytd_year_date < fye_year_date:
  revenue = float(income_statement.get('annualReports', [])[0].get('totalRevenue'))
  gross_profit = float(income_statement.get('annualReports', [])[0].get('grossProfit'))
  operating_expenses = float(income_statement.get('annualReports', [])[0].get('operatingExpenses'))
  depreciation_and_amortization = float(income_statement.get('annualReports', [])[0].get('depreciationAndAmortization'))
  net_income = float(income_statement.get('annualReports', [])[0].get('netIncome'))
  tax = float(income_statement.get('annualReports', [])[0].get('incomeTaxExpense'))
  tax_rate = tax/(net_income+tax)
  shares_outstanding = float(balance_sheet.get('annualReports', [])[0].get('commonStockSharesOutstanding'))
  Assets = float(balance_sheet.get('annualReports', [])[0].get("totalAssets"))
  Debt = float(balance_sheet.get('annualReports', [])[0].get("totalLiabilities"))

else:
  # Calculate trailing twelve months (TTM) values
  revenue = float(income_statement.get('quarterlyReports', [])[0].get('totalRevenue', 0)) + \
            float(income_statement.get('quarterlyReports', [])[1].get('totalRevenue', 0)) + \
            float(income_statement.get('quarterlyReports', [])[2].get('totalRevenue', 0)) + \
            float(income_statement.get('quarterlyReports', [])[3].get('totalRevenue', 0))

  gross_profit = float(income_statement.get('quarterlyReports', [])[0].get('grossProfit', 0)) + \
                float(income_statement.get('quarterlyReports', [])[1].get('grossProfit', 0)) + \
                float(income_statement.get('quarterlyReports', [])[2].get('grossProfit', 0)) + \
                float(income_statement.get('quarterlyReports', [])[3].get('grossProfit', 0))

  operating_expenses = float(income_statement.get('quarterlyReports', [])[0].get('operatingExpenses', 0)) + \
                      float(income_statement.get('quarterlyReports', [])[1].get('operatingExpenses', 0)) + \
                      float(income_statement.get('quarterlyReports', [])[2].get('operatingExpenses', 0)) + \
                      float(income_statement.get('quarterlyReports', [])[3].get('operatingExpenses', 0))

  # Note: Depreciation and Amortization might be nested under another key.
  # Adjust the key path accordingly.
  depreciation_and_amortization = float(income_statement.get('quarterlyReports', [])[0].get('depreciationAndAmortization', 0)) + \
                                  float(income_statement.get('quarterlyReports', [])[1].get('depreciationAndAmortization', 0)) + \
                                  float(income_statement.get('quarterlyReports', [])[2].get('depreciationAndAmortization', 0)) + \
                                  float(income_statement.get('quarterlyReports', [])[3].get('depreciationAndAmortization', 0))

  net_income = float(income_statement.get('quarterlyReports', [])[0].get('netIncome', 0)) + \
              float(income_statement.get('quarterlyReports', [])[1].get('netIncome', 0)) + \
              float(income_statement.get('quarterlyReports', [])[2].get('netIncome', 0)) + \
              float(income_statement.get('quarterlyReports', [])[3].get('netIncome', 0))

  tax = float(income_statement.get('quarterlyReports', [])[0].get('incomeTaxExpense', 0)) + \
        float(income_statement.get('quarterlyReports', [])[1].get('incomeTaxExpense', 0)) + \
        float(income_statement.get('quarterlyReports', [])[2].get('incomeTaxExpense', 0)) + \
        float(income_statement.get('quarterlyReports', [])[3].get('incomeTaxExpense', 0))

  Assets = float(balance_sheet.get('quarterlyReports', [])[0].get("totalAssets"))
  Debt = float(balance_sheet.get('quarterlyReports', [])[0].get("totalLiabilities"))

  # Calculate tax rate
  if net_income != 0:
      tax_rate = tax / net_income
  else:
      tax_rate = 0  # Handle cases where net income is zero

  # Shares outstanding might be in the balance sheet or a separate API call.
  # Adjust the data source and key accordingly.
  shares_outstanding = float(balance_sheet.get('quarterlyReports', [])[0].get('commonStockSharesOutstanding', 0))


In [31]:
difference = ytd_year_date - fye_year_date

# Calculate total difference in months
total_months = (difference.days / 30)  # Approximate conversion (30 days per month)

# Round to the nearest multiple of 3
rounded_months = round(total_months / 3) * 3

in_years = rounded_months / 12
in_years

0.5

In [44]:
# Calculate the future growth rates
future_revenue_growth = float(input(f"Enter the future revenue growth rate"))
future_gross_margin = float(input(f"Enter the future gross margin"))
future_opex_margin = float(input(f"Enter the future opex margin"))
future_d_a_margin = float(input(f"Enter the future d&a margin"))
future_capex_margin = float(input(f"Enter the future capex margin"))
future_net_working_capital_change_percentage = float(input(f"Enter the future net working capital change percentage"))
future_tax_rate = float(input(f"Enter the future tax rate"))
#Get current share price
current_share_price = float(input(f"Enter the current share price "))

# Calculate the WACC

#Calculate cost of equity
risk_free_rate = float(input(f"Enter the 10-zear treasury rate"))
df_ERP = pd.read_excel("https://github.com/ConstantinVictorBeatErtel/btflyfi/raw/main/histimpl.xls", skiprows=6)
df_ERP = df_ERP.dropna(subset=["Year"]).iloc[:-1, :].set_index("Year")
ERP = df_ERP["Implied ERP (FCFE)"].values[-1]
beta = future_tax_rate
CostOfEquity = beta*(ERP) + risk_free_rate

#calculate cost of debt
ytm = float(input(f"Enter the yield to maturity"))

#Calculate WACC
market_cap = float(input(f"Enter the market cap"))


total = market_cap + Debt
AfterTaxCostOfDebt = ytm * (1-tax_rate)
wacc = (AfterTaxCostOfDebt*Debt/total) + (CostOfEquity*market_cap/total)

# Build the discounted cash flow model
discount_rate = wacc
terminal_growth_rate = float(input(f"Enter the terminal growth rate"))
periods = int(input(f"Enter the number of periods"))

Enter the future revenue growth rate0.11
Enter the future gross margin0.895
Enter the future opex margin0.43
Enter the future d&a margin0.04
Enter the future capex margin0.01
Enter the future net working capital change percentage0.1
Enter the future tax rate0.15
Enter the 10-zear treasury rate0.0437
Enter the future tax rate0.15
Enter the yield to maturity0.05
Enter the market cap256950000000
Enter the terminal growth rate0.02
Enter the number of periods5


In [69]:
# Initialize lists to store data for Excel
years = []
revenues = []
gross_profits = []
operating_expenses = []
d_and_a = []
ebit = []
taxes = []
net_incomes = []
capex = []
cash_flows = []
discount_factors = []
present_values = []

# Calculate the present value of future cash flows and store data for each year
present_value = 0
for i in range(periods):
  year = i + 1

  #discount_factor = 1 / (1 + discount_rate) ** (year - in_years)

  #present_value_year = cash_flow * discount_factor
  #present_values.append(present_value_year)

  revenue_projected = revenue * (1 + future_revenue_growth)**i
  gross_profit_projected = revenue_projected * future_gross_margin
  operating_expense_projected = revenue_projected * future_opex_margin
  d_and_a_expense_projected = revenue_projected * future_d_a_margin
  ebit_projected = gross_profit_projected - operating_expense_projected
  tax_projected = ebit_projected * future_tax_rate
  net_income_projected = ebit_projected - tax_projected
  capex_expense_projected = revenue_projected * future_capex_margin
  future_net_working_capital_change = net_income_projected * future_net_working_capital_change_percentage
  cash_flow = net_income_projected - capex_expense_projected + d_and_a_expense_projected - future_net_working_capital_change
  discount_factor = 1 / (1 + discount_rate)**year
  present_value_year = cash_flow * discount_factor
  present_value += present_value_year

  # Append data to lists
  years.append(year)
  revenues.append(revenue_projected)
  gross_profits.append(gross_profit_projected)
  operating_expenses.append(operating_expense_projected)
  d_and_a.append(d_and_a_expense_projected)
  ebit.append(ebit_projected)
  taxes.append(tax_projected)
  net_incomes.append(net_income_projected)
  capex.append(capex_expense_projected)
  cash_flows.append(cash_flow)
  discount_factors.append(discount_factor)
  present_values.append(present_value_year)

# Calculate the terminal value
terminal_value = cash_flows[-1] * (1 + terminal_growth_rate) / (discount_rate - terminal_growth_rate)
present_value += terminal_value / (1 + discount_rate)**periods

# Calculate the enterprise value
enterprise_value = present_value/shares_outstanding



#Calculate the upside
upside = (enterprise_value/current_share_price - 1) * 100

# Create a Pandas DataFrame from the calculated data

data = {
    'Revenue': revenues,
    'Gross Profit': gross_profits,
    'Operating Expenses': operating_expenses,
    'D&A': d_and_a,
    'EBIT': ebit,
    'Taxes': taxes,
    'Net Income': net_incomes,
    'Capex': capex,
    'Cash Flow': cash_flows,
    'Discount Factor': discount_factors,
    'Present Value': present_values
}

df = pd.DataFrame(data, index=years).transpose()

num_columns = len(df.columns)

# Add Terminal Value row
df.loc['Terminal Value'] = [''] * (num_columns - 1) + [terminal_value / (1 + discount_rate)**periods]

# Add Enterprise Value row
df.loc['Enterprise Value'] = [''] * (num_columns - 1) + [enterprise_value]

# Add Upside row
df.loc['Upside'] = [''] * (num_columns - 1) + [upside]

# Reset the index to make column labels the first row
df = df.reset_index()

# Export the DataFrame to an Excel file
df.to_excel("dcf_calculations.xlsx", index=False)

